Notebook ini dibuat sesederhana mungkin, mengikuti petunjuk garis besar yang tedapat pada tautan [berikut](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html). Karena permasalahan yang dihadapi adalah regresi (memprediksi suatu nilai) dengan ukuran sampel lebih dari 100 ribu, dipilihlah estimator SGDRegressor.

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDRegressor

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/train.csv').drop(columns = 'id')
test = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/test.csv').drop(columns = 'id')
submission = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv')

Kolom kategorikal pada dataframe train maupun test bisa mencerminkan suatu kategori ordinal maupun nonordinal. Pada percobaan kali ini, kolom kategorikal dianggap bersifat nonordinal, sehingga akan disandikan dengan metode one hot encoder.

In [ ]:
X = pd.get_dummies(train.drop(columns = 'target'),drop_first = True).drop(columns = 'cat6_G')
y = train['target']
test = pd.get_dummies(test,drop_first = True)

Langkah selanjutnya adalah melatih model dan sekaligus memvalidasi keumumannya menggunakan metode cross validation. Akurasi model diukur menggunakan root mean squared error (RMSE) sebagaimana metrics yang digunakan oleh platfom kaggle untuk kompetensi ini.

In [ ]:
regr = SGDRegressor(random_state = 42)
scores = cross_val_score(regr,X,y,scoring = 'neg_root_mean_squared_error')
print(f'Rerata skor RMSE {-scores.mean()} dengan simpangan baku {scores.std()}.')

Menimbang bahwa model memiliki keumuman yang layak berdasarkan hasil cross validation, maka model dapat dilatih menggunakan keseluruhan dataframe train.

In [ ]:
regr = regr.fit(X,y)
submission['target'] = pd.Series(regr.predict(test))

In [ ]:
submission.set_index('id').to_csv('submission.csv')

Kualitas prediksi dengan teknik seperti di atas ternyata menghasilkan skor publik RMSE sebesar 0.86415, hanya sedikit lebih buruk dibanding skor peringkat 1 sebesesar 0.84115. Ini menunjukkan bahwa pustaka sklearn dengan kode minimal dan parameter bawaan sebenarnya sudah sangat baik dalam melakukan tugasnya.